# Polynomial Regression
So according to the visualizations from the <a href="https://github.com/lynstanford/machine-learning-projects/machine-learning/multiple_regression.ipynb">Linear Regression</a> notebook, the connection between the dependent and independent variables appear to be strongly linear in relationship although the connection between the 'Volume' of daily bitcoin bought and sold has less association with the daily 'Close' price. 

Fitting a Linear Regression line to the data may be accurate in this case, with an R2 value of 0.9991392014437468 and RMSE of 689.1925598643533. However, out of curiosity I decided to see if a Polynomial function could fit the line slightly better by employing a regularization technique to try and improve the bias term by decreasing the Mean Squared Error.

The r-squared value is used to represent the overall accuracy score and directly measures the degree of variability associated between the predictors and target variable. The root mean squared value is represented as a loss function and my aim is to reduce its overall value as much as possible using regularization.

## Import Data
Keeping the data loading simple this time will reduce the overall time it takes to retrieve.

In [ ]:
# import libraries
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# import data
btc_cad = pd.read_csv("C:/Users/lynst/Documents/GitHub/machine-learning-projects/machine-learning/BTC_CAD.csv")
bitcoin = pd.DataFrame(btc_cad).dropna(axis=0)
bitcoin

## Feature Selection
Establishing the features contained within this dataset with empty rows already removed:

In [ ]:
# all column names
bitcoin.columns

Next, I want to determine what data type each column has before I start my feature selection and pre-processing.

In [ ]:
# all column data types
bitcoin.dtypes

Assign the (dependant) y variable and (independent) X variables for the modelling process. Once again, I've decided to remove the Date and Adjusted Close columns and to use the Close price as the target variable.

## Feature Selection and Normalization
Having read the data in from the filepath I'd like to store it in a dataframe.

In [ ]:
import pandas as pd
from sklearn import preprocessing
import os

# Location of the file path
bitcoin = pd.read_csv("C:/Users/lynst/Documents/GitHub/machine-learning-projects/machine-learning/BTC_CAD.csv")

# Open the file into a variable called df
df = pd.DataFrame(bitcoin).dropna(axis=0)

In [ ]:
# print columns in dataset
df.columns

This provides the entire dataset with all the column headers and their values converted to a single dataframe. Now to remove the columns I don't need including 'Date', 'Adj Close', selecting only the remaining columns I want to include in the X variable such as 'Open', 'High', 'Low' and 'Volume' and the remaining column of 'Close' in the same dataframe will be used as the target variable output, y.

In [ ]:
# remove feature with string values - Date
del bitcoin['Date']

# remove adj close as I will not be using this
del bitcoin['Adj Close']

# set the feature columns list
bitcoin_features = ['Open','High','Low','Volume']
print(bitcoin_features)

Comparing the shape of the overall dataset before polynomial regression and before splitting gives:

In [ ]:
# select features as X dataframe
X = df[bitcoin_features]
print(X)

In [ ]:
X.shape

In [ ]:
# select target as y series
y = df['Close']
print(y)

In [ ]:
y.shape

Another way to accomplish this would be to specify the columns to be used for X:

In [ ]:
# select the first ten entries
X = bitcoin[["Open", "High", "Low", "Volume"]].dropna(axis=0)
print(X[0:10])

And for y:

In [ ]:
# print the first ten values
y = bitcoin[["Close"]].dropna(axis=0)
print(y[0:10])

Does the dataset need re-scaling? In this particular case the (X) predictors include 3 price variables and 1 volume variable which is scaled differently. The (y) target variable is another price variable, so the data would benefit from normalization. This transformation will shift the values to a range between 0 and 1 for each column. 

In [ ]:
# assign bitcoin dataset to df
df = bitcoin.dropna(axis=0)

# convert entries to float type
X = df.values.astype(float)

# define min max scaler
min_max_scaler = preprocessing.MinMaxScaler()

# transform data
X_scaled = min_max_scaler.fit_transform(X)

df = pd.DataFrame(X_scaled, columns=df.columns)
print(df)

Make a copy of the dataframe first.

In [ ]:
bitcoin = bitcoin.copy()

In [ ]:
X = df[bitcoin_features]
X.shape

Check to see if the values have been re-scaled properly by calling the .head method:

In [ ]:
X.head

In [ ]:
y.shape

## Split the Dataset
First I've decided to import the train_test_split function to split the dataset into respective training and test sets (for both X and y values in each set).

In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

## Train Linear Regression Model
Train the linear model on the data and fit a regression line.

In [ ]:
# Fitting Linear Regression to the dataset
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

## Predicting Against Test Set
Now to predict the test set results.

In [ ]:
y_pred = lin_reg.predict(X_test)

# to see the first ten results
y_pred[0:10]

Comparing the first row of X values to predict the y value by copying from the dataset output from cell 1. Remember only to use the 'Open', 'High', 'Low' and 'Volume' values.

In [ ]:
lin_reg.predict([[9718.07, 9838.33, 9728.25, 4.624843e+10]])

So the predicted y value is 9777.86 and the actual y value (from output of cell 1 for 'Close' price) is 9763.94, which is reasonably similar.

## Model Evaluation
This is where I need to look into the root mean squared error and R-squared values to see how well the model is performing.

In [ ]:
# import the score measure
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

## Visualization
Plotting the actual and predicted results for the 'Close' price and fitting a regression line provides us with a nice visual display.

In [ ]:
plt.figure(figsize=(15,10))
plt.scatter(y_test, y_pred)
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.title("Actual vs. Predicted Prices")

## Predictions
Storing the actual and predicted values and their difference in a dictionary object becomes useful. The larger the error, the greater the variance and subsequent standard deviation which translates to less accurate predictions and a less reliable model. 

Why does this become important? The main purpose of this exercise is to see how useful my model would be in predicting the daily closing price so the first information I can derive from the scatter plot above shows the relationship between independent and dependent variables is strong, but also incredibly linear. It may not pay off to apply Polynomial Regression in such circumstances but may improve the linear fit to the data points with increasing degrees of freedom.

In [ ]:
predictions = pd.DataFrame({"Actual": y_test, "Predicted": y_pred, "Error": y_test - y_pred})
predictions[0:20]

Exploring the entire dataset (all the X and y values) and the degree of accuracy by fitting a linear regression predictive line once more:

In [ ]:
# converting the dictionary value objects
X = predictions[y_test].values
y = predictions[y_pred].values

# Fitting Linear Regression to the dataset
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X, y)

# Visualizing the Linear Regression results
import matplotlib.pyplot as plt
def viz_linear():
    plt.scatter(X, y, color='red')
    plt.plot(X, lin_reg.predict(X, color='blue')
    plt.title('Actual vs. Predicted Bitcoin Prices')
    plt.xlabel('Actual Price')
    plt.ylabel('Predicted Price')
    plt.show()
    return
viz_linear()

In [ ]:
import seaborn as sns

# set the width and height of the plot
plt.figure(figsize=(7,7))

# visualizing the relationship between actual and predicted values for y
plt.scatter(y_test, price_predictions)
plt.xlabel("Actual Prices")
plt.ylabel("Predicted Prices")
plt.title("Actual Prices vs Predicted Prices")

# fit a regression line between high and low values to show linear nature
sns.regplot(x=y_test, y=price_predictions)

In [ ]:

'''
# Visualising the Linear Regression results
plt.scatter(X, y, color = 'blue')
  
plt.plot(X, lin.predict(X), color = 'red')
plt.title('Linear Regression')
plt.xlabel('Temperature')
plt.ylabel('Pressure')
  
plt.show()'''

## Training the Polynomial model
Importing the relevant libraries and instantiating the polynomial function model followed by splitting the data into training and validation sets. The first stage will produce an expanded feature set for the entire dataframe with quadratic terms (degree=2). I will attempt to tune this hyperparameter later to see if it can optimize the Loss function.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

# initialize the model for a given degree
poly_features = PolynomialFeatures(degree=2, include_bias=True)
  
# transforms the existing features to higher degree features.
poly_X = poly_features.fit_transform(X)

Using indexation to return any value in X, say the 1st value

In [ ]:
X[:1]

Now repeating for the data contained in the 'poly_X' set and we can see that there are 15 values returned so it has created an array with 11 new features for a total of 15 features. This includes element-wise dot product values and some squared values (without going into too much detail). Both the original feature values for 'X1' to 'Xn' and the feature squared value from 'poly_X' are returned in this example.

In [ ]:
poly_X[:1]

Saving the current dataframe to a CSV formatted file (or an Excel file) for preservation to view the degree of newly expanded features in a fresh table.

In [ ]:
df.to_csv(r'C:/Users/lynst/Documents/GitHub/machine-learning-projects/machine-learning/X_train_poly.csv', index=False, header=True)

### Splitting the Data
Using this data for the Polynomial Features model and splitting it into training and test sets with a 70-30 split. 

In [ ]:
# then split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(poly_X, y, test_size = 0.3, random_state = 42)

The number of entries (rows) by the number of features (cols):

In [ ]:
print(X.shape)
print(y.shape)

But the shape of the new dataset with all the extended features is much larger.

In [ ]:
print(poly_X.shape)

Printing out the shape of the training set from the polynomial data gives:

In [ ]:
print(X_train.shape)
print(y_train.shape)

And the shape of the test data:

In [ ]:
print(X_test.shape)
print(y_test.shape)

### Applying Linear Regression First

In [ ]:
from sklearn.linear_model import LinearRegression

# fit the transformed features to Linear Regression
linear_regression = LinearRegression()

# fit the model to training set only!!!
linear_regression.fit(poly_X, y)

### Making a Prediction
Comparing actual and predicted 'Close' prices between labelled and target data in the test set tells me that the accuracy isn't great and there seems to be considerable variance.

In [ ]:
poly_y_pred = linear_regression.predict(poly_X)
#predictions = pd.DataFrame(poly_y_pred)

#print("Predictions: ", linear_regression.predict(poly_X.iloc[:4]))
#print("Actual: ", poly_X.iloc[:1])


In [ ]:
print(y.shape)

Comparing predictions for training set and test set values

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import seaborn as sns

mse = mean_squared_error(y_test, y_test_pred)
print(mse)

rmse = np.sqrt(mse)
print(rmse)

# set the width and height of the plot
plt.figure(figsize=(6,6))

# visualizing the relationship between actual and predicted values for y
plt.scatter(y_test, y_test_pred)
plt.xlabel("Actual Prices")
plt.ylabel("Predicted Prices")
plt.title("Actual Prices vs Predicted Prices")

# fit a regression line between high and low values to show linear nature
sns.regplot(x=y_test, y=y_test_pred)

# evaluating the model on training dataset
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
r2_train = r2_score(y_train, y_train_pred)
  
# evaluating the model on test dataset
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)

print("\n")

print("The model performance for the training set")
print("-------------------------------------------")
print("RMSE of training set is {}".format(rmse_train))
print("R2 score of training set is {}".format(r2_train))
  
print("\n")
  
print("The model performance for the test set")
print("-------------------------------------------")
print("RMSE of test set is {}".format(rmse_test))
print("R2 score of test set is {}".format(r2_test))

print("\n")

Defining the intercept of the polynomial line and coefficient values for x1, x2 ... xn.

In [ ]:
ridge_regression.intercept_
ridge_regression.coef_

Given:

y = a + b1x1 + b2x2 + b3x3 + b4x4

### Utilizing Regularization
In this particular instance I have chosen L2 regularization which will penalize the Loss (or cost) function by summing the coefficients and equalizing features which exhibit strong colinearity, while simultaneously reducing the effect of redundant coefficients. (See the colinearity matrix in the previous <a href="https://github.com/lynstanford/machine-learning-projects/machine-learning/multiple_regression.ipynb">Linear Regression</a> model).
    
Note, I have two hyperparameters I can adjust here including 'alpha' and 'normalize'. The 'alpha' value is the learning rate which controls the size of the incremental steps for the learning rate and the 'normalize' adjustment will re-scale the feature values between 0 and 1 for each observation in the dataset.

In [ ]:
from sklearn import linear_model

# initialize ridge regression
ridge_regression = linear_model.Ridge(alpha=0.1, normalize=True)

# fit the model to training set only!!!
ridge_regression.fit(X_train, y_train)

The regularization term should only be added to the cost function during the training phase. Now the 'training' data has been fit, it becomes important to discover the performance measure on the unregularized test set. Making a prediction on the first 5 values in the test set first.

In [ ]:
y_test_pred = ridge_regression.predict(X_test)
print("Predictions: ", ridge_regression.predict(X_test.iloc[:5]))

Defining the intercept of the polynomial line and coefficient values for x1, x2 ... xn.

In [ ]:
ridge_regression.intercept_
ridge_regression.coef_

Given:

    y = a + b1x1 + b2x2 + b3x3 + b4x4

This tells me the prediction for the target output variable, y, based on the input variables specified and using ridge regression giving the value of C$37,990.99.

## Model Validation Metrics¶
Now to measure the error score and accuracy of the line of fit.

## Applying Regularization
To avoid overfitting regularization can be applied, but will require tuning hyperparameters manually to an extent. To see if I can improve on the linear regression model's ability to predict the target variable, I have decided to use Ridge Regression.
This regularization term basically employs the use of a penalization method by summing the squared values of each coefficient (whether positive or negative) and helps balance the contribution of all the features but this term is only used during the training phase so I need to remove it for the purpose of testing and evaluation.

## Model Selection
### Ridge Regression
Trying a slightly different type of regression model using the alpha learning rate hyperparameter of 1.0 to see if I can reduce the (rmse) error value and increase the (r2) accuracy score. The purpose of using a ridge regression model is to try to reduce or eliminate the coefficient values of all the various features (especially those with high multi-colinearity between predictors) and will increase bias slightly but should decrease variance significantly. Ridge regression achieves this by assigning equal weights to those coefficient values which have high colinearity.

In [ ]:
# instantiate model
ridge_regression = linear_model.Ridge(alpha=1.0)

# fit model
ridge_regression.fit(X_train, y_train)

The regularization term should only be added to the cost function during the training phase. Now the 'training' data has been fit, it becomes important to discover the performance measure on the unregularized test set. Making a prediction on the first 5 values in the test set first.

In [ ]:
price_predictions = ridge_regression.predict(X_test)
print("Predictions: ", ridge_regression.predict(X_test.iloc[:5]))

This tells me the prediction for the target output variable, y, based on the input variables specified and using ridge regression giving the value of C$37,990.99.

## Model Validation Metrics¶
Now to measure the error score and accuracy of the line of fit.

This new data matrix containing the additional features with the squared values has been created by expanding the number of features and the parameter weights (or coefficients) which represent a quadratic equation. The linear regression model should now be applied again to this newly expanded dataframe containing the polynomial features.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, price_predictions)
print(mse)

rmse = np.sqrt(mse)
print(rmse)

# set the width and height of the plot
plt.figure(figsize=(6,6))

# visualizing the relationship between actual and predicted values for y
plt.scatter(y_test, price_predictions)
plt.xlabel("Actual Prices")
plt.ylabel("Predicted Prices")
plt.title("Actual Prices vs Predicted Prices")

# fit a regression line between high and low values to show linear nature
sns.regplot(x=y_test, y=price_predictions)

In [ ]:
print("R-squared: ", ridge_regression.score(X_test, y_test))

Producing a scatter plot of the data points to display actual vs predicted prices and the regression line of best fit.

In [ ]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, price_predictions)
print(mse)

rmse = np.sqrt(mse)
print(rmse)

# set the width and height of the plot
plt.figure(figsize=(6,6))

# visualizing the relationship between actual and predicted values for y
plt.scatter(y_test, price_predictions)
plt.xlabel("Actual Prices")
plt.ylabel("Predicted Prices")
plt.title("Actual Prices vs Predicted Prices")

# fit a regression line between high and low values to show linear nature
sns.regplot(x=y_test, y=price_predictions)

In [ ]:
print(ridge_regression.intercept_)
print(ridge_regression.coef_)

This is a slight improvement on the original R-squared score and I can see that the RMSE hasn't really changed.
Trying to improve on the scores above, I decided to introduce a Polynomial model which should be able to fit the line more accurately to the data points.

## Using a Pipeline
The next method involves placing the expanded polynomial features and linear regression of these within a pipeline which can be trained and used to predict the target variable.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

pipeline = Pipeline([
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('reg', LinearRegression(normalize=True))
    ])

pipeline.fit(X_train, y_train)
y_train_pred = pipeline.predict(X_train)
print(y_train_pred)

Next I wanted to find the shape of the matrices involved and now these datasets have been expanded by the polynomial model I want to make sure they have the same dimensions (m.n) otherwise the polynomial regression won't work.

In [ ]:
print(X_train.shape)

In [ ]:
print(y_train.shape)

In [ ]:
print(y_train_pred.shape)

In [ ]:
# evaluating the model on training dataset
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
r2_train = r2_score(y_train, y_train_pred)

# print the RMSE metric and R2 accuracy score
print("RMSE of training set is {}".format(rmse_train))
print("R2 score of training set is {}".format(r2_train))

The metrics from using the 2nd order quadratic coefficients and terms on the training set, a significant improvement.

In [ ]:
pipeline = Pipeline([
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('reg', LinearRegression(normalize=True))
    ])

pipeline.fit(X_test, y_test)
y_test_pred = pipeline.predict(X_test)
print(y_test_pred)

The metrics from generalizing to the test set data for the quadratic equation.

In [ ]:
# evaluating the model on test dataset
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)

# print the RMSE metric and R2 accuracy score
print("RMSE of test set is {}".format(rmse_test))
print("R2 score of test set is {}".format(r2_test))

In [ ]:
pipeline = Pipeline([
    ('poly', PolynomialFeatures(degree=3, include_bias=False)),
    ('reg', LinearRegression(normalize=True))
    ])

pipeline.fit(X_train, y_train)
y_train_pred = pipeline.predict(X_train)
print(y_train_pred)

# evaluating the model on training dataset
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
r2_train = r2_score(y_train, y_train_pred)

# print the RMSE metric and R2 accuracy score
print("\n")
print("RMSE of training set is {}".format(rmse_train))
print("R2 score of training set is {}".format(r2_train))

In [ ]:
print(X_test.shape)

In [ ]:
print(y_test.shape)

In [ ]:
print(y_test_pred.shape)

In [ ]:
pipeline = Pipeline([
    ('poly', PolynomialFeatures(degree=3, include_bias=False)),
    ('reg', LinearRegression(normalize=True))
    ])

pipeline.fit(X_test, y_test)
y_test_pred = pipeline.predict(X_test)
print(y_test_pred)

# evaluating the model on test dataset
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
r2_test = r2_score(y_test, y_test_pred)

# print the RMSE metric and R2 accuracy score
print("\n")
print("RMSE of test set is {}".format(rmse_test))
print("R2 score of test set is {}".format(r2_test))

The metrics from using an equation with 3rd order cubic coefficients and terms on the test set produced the best overall scores so far.

# Model Validation
So the fit of the regression forecast line to the data has increased in its accuracy marginally with the degree of fit (or expansion of terms) and the RMSE has reduced fairly considerably. The R-squared accuracy appears to increase with smaller sets of data such as the test sets overall.

Ridge regression rmse score = 689.1925582115692
R-squared accuracy = 0.9991392014478754

So I can determine the use of ridge regression to regularize the model has produced a slight increase in overall variance but also a slight increase in bias of output prediction. Ridge regression should reduce the variance and if not, I probably need to adjust the 'degrees of freedom' hyperparameter for the cost function for the Polynomial model which feeds into it, or tune the 'alpha' hyperparameter in the actual ridge regression model itself. Tuning these inputs ...........

I have decided to see if I can improve the model's predictive power by electing to use a Decision Tree Regression model: "https://github.com/lynstanford/machine-learning-projects/tree/master/machine-learning/decision_tree.ipynb".

# Print dependencies
Dependencies are fundamental to record the **computational environment**.   

- Use [watermark](https://github.com/rasbt/watermark) to print version of python, ipython, and packages, and characteristics of the computer

In [ ]:
%load_ext watermark

# python, ipython, packages, and machine characteristics
%watermark -v -m -p wget,pandas,numpy,watermark,tarfile,urllib3,matplotlib,seaborn,sklearn,pickle5

# date
print (" ")
%watermark -u -n -t -z 